In [2]:
import QuantLib as ql

# Analytic continuous geometric average-price Asians

In [3]:
dc = ql.Actual360()

In [4]:
today = ql.Settings.instance().evaluationDate

In [9]:
spot = ql.SimpleQuote(80.0)
qRate = ql.SimpleQuote(-0.03)
qTS = ql.FlatForward(today, ql.QuoteHandle(qRate), dc)
rRate = ql.SimpleQuote(0.05)
rTS = ql.FlatForward(today, ql.QuoteHandle(rRate), dc)
vol = ql.SimpleQuote(0.20)
volTS = ql.BlackConstantVol(today, ql.NullCalendar(), ql.QuoteHandle(vol), dc)

In [11]:
stochProcess = ql.BlackScholesMertonProcess(ql.QuoteHandle(spot), ql.YieldTermStructureHandle(qTS), 
                                            ql.YieldTermStructureHandle(rTS), 
                                            ql.BlackVolTermStructureHandle(volTS))

In [12]:
engine = ql.AnalyticContinuousGeometricAveragePriceAsianEngine(stochProcess)

In [13]:
averageType = ql.Average.Geometric

In [15]:
option_type = ql.Option.Put

In [16]:
strike = 85.0
exerciseDate = today + 90

In [17]:
pastFixings = ql.nullInt()

In [18]:
runningAccumulator = ql.nullDouble()

In [20]:
payoff = ql.PlainVanillaPayoff(option_type, strike)

In [21]:
exercise = ql.EuropeanExercise(exerciseDate)

In [22]:
option = ql.ContinuousAveragingAsianOption(averageType, payoff, exercise)

In [23]:
option.setPricingEngine(engine)

In [24]:
calculated = option.NPV()

In [25]:
expected = 4.6922

In [26]:
tolerance = 1.0e-4

In [27]:
if (abs(calculated-expected) > tolerance):
    print("FAILURE")

# Analytic continuous geometric average-price Asian greeks

In [30]:
calculated, expected, tolerance = {}, {}, {}

In [31]:
tolerance["delta"]  = 1.0e-5
tolerance["gamma"]  = 1.0e-5
tolerance["theta"]  = 1.0e-5
tolerance["rho"]    = 1.0e-5
tolerance["divRho"] = 1.0e-5
tolerance["vega"]   = 1.0e-5

In [32]:
types = [ql.Option.Call, ql.Option.Put]

In [33]:
underlyings = [100.0]

In [34]:
strikes = [90.0, 100.0, 110.0]

In [35]:
qRates = [0.04, 0.05, 0.06]

In [37]:
rRates = [0.01, 0.05, 0.15]

In [38]:
lengths = [1, 2]

In [39]:
vols = [0.11, 0.50, 1.20]

In [40]:
dc = ql.Actual360()

In [41]:
today = ql.Settings.instance().evaluationDate

In [42]:
ql.Settings.instance().evaluationDate = today

In [82]:
spot = ql.SimpleQuote(0.0)
qRate = ql.SimpleQuote(0.0)
qTS = ql.YieldTermStructureHandle(ql.FlatForward(0, ql.NullCalendar(), ql.QuoteHandle(qRate), dc))
rRate = ql.SimpleQuote(0.0)
rTS = ql.YieldTermStructureHandle(ql.FlatForward(0, ql.NullCalendar(), ql.QuoteHandle(rRate), dc))
vol = ql.SimpleQuote(0.0)
volTS = ql.BlackVolTermStructureHandle(ql.BlackConstantVol(0, ql.NullCalendar(), ql.QuoteHandle(vol), dc))

In [83]:
process = ql.BlackScholesMertonProcess(ql.QuoteHandle(spot), qTS, 
                                            rTS, 
                                            volTS)

In [84]:
def relativeError(x1, x2, reference):
        if (reference != 0.0):
            return abs(x1-x2)/reference
        else:
            # fall back to absolute error
            return abs(x1-x2)

In [85]:
def REPORT_FAILURE(greekName, averageType, runningAccumulator, pastFixings,
                       fixingDates, payoff, exercise, s, q, r, today, v,
                       expected, calculated, tolerance):
    print (str(exercise)
                                        + " Asian option with "
                                        + str(averageType) + " and "
                                        + str(payoff) + " payoff:\n"

                                        + "    future fixings:   " + str(len(fixingDates)) + "\n"
                                        + "    underlying value: " + str(s) + "\n"
                                        + "    strike:           " + str(payoff.strike()) + "\n"
                                        + "    dividend yield:   " + str(q) + "\n"
                                        + "    risk-free rate:   " + str(r) + "\n"
                                        + "    reference date:   " + str(today) + "\n"
                                        + "    maturity:         " + str(exercise.lastDate()) + "\n"
                                        + "    volatility:       " + str(v) + "\n\n"
                                        + "    expected   " + greekName + ": " + str(expected) + "\n"
                                        + "    calculated " + greekName + ": " + str(calculated) + "\n"
                                        + "    error:            " + str(abs(expected-calculated))
                                        + "\n"
                                        + "    tolerance:        " + str(tolerance))

In [93]:
2*ql.Years

TypeError: 'int' object is not callable

In [90]:
today + length*ql.Years


Date(19,4,2019)

In [94]:
for option_type in types:
    for strike in strikes:
        for length in lengths:
            maturity = ql.EuropeanExercise(today + ql.Period(length,ql.Years))
            payoff = ql.PlainVanillaPayoff(option_type, strike)
            engine = ql.AnalyticContinuousGeometricAveragePriceAsianEngine(process)
            option = ql.ContinuousAveragingAsianOption(ql.Average.Geometric, payoff, maturity)
            option.setPricingEngine(engine)
            pastFixings = ql.nullInt()
            runningAccumulator = ql.nullDouble()
            for u in underlyings:
                for q in qRates:
                    for r in rRates:
                        for v in vols:
                            spot.setValue(u)
                            qRate.setValue(q)
                            rRate.setValue(r)
                            vol.setValue(v)
                            value = option.NPV()
                            calculated["delta"]  = option.delta()
                            calculated["gamma"]  = option.gamma()
                            calculated["theta"]  = option.theta()
                            calculated["rho"]    = option.rho()
                            calculated["divRho"] = option.dividendRho()
                            calculated["vega"]   = option.vega()
                            if (value > spot.value()*1.0e-5):
                                du = u*1.0e-4
                                spot.setValue(u+du)
                                value_p = option.NPV()
                                delta_p = option.delta()
                                spot.setValue(u-du)
                                value_m = option.NPV()
                                delta_m = option.delta()
                                spot.setValue(u)
                                expected["delta"] = (value_p - value_m)/(2*du)
                                expected["gamma"] = (delta_p - delta_m)/(2*du)
                                
                                dr = r*1.0e-4
                                rRate.setValue(r+dr)
                                value_p = option.NPV()
                                rRate.setValue(r-dr)
                                value_m = option.NPV()
                                rRate.setValue(r)
                                expected["rho"] = (value_p - value_m)/(2*dr)
                                
                                dq = q*1.0e-4
                                qRate.setValue(q+dq)
                                value_p = option.NPV()
                                qRate.setValue(q-dq)
                                value_m = option.NPV()
                                qRate.setValue(q)
                                expected["divRho"] = (value_p - value_m)/(2*dq)
                                
                                dv = v*1.0e-4
                                vol.setValue(v+dv)
                                value_p = option.NPV()
                                vol.setValue(v-dv)
                                value_m = option.NPV()
                                vol.setValue(v)
                                expected["vega"] = (value_p - value_m)/(2*dv)
                                
                                dT = dc.yearFraction(today-1, today+1)
                                ql.Settings.instance().evaluationDate = today - 1
                                value_m = option.NPV()
                                ql.Settings.instance().evaluationDate = today + 1
                                value_p = option.NPV()
                                ql.Settings.instance().evaluationDate = today
                                expected["theta"] = (value_p - value_m)/dT
                                
                                for greek in calculated.keys():
                                    expct = expected  [greek]
                                    calcl = calculated[greek]
                                    tol   = tolerance [greek]
                                    error = relativeError(expct,calcl,u)
                                    
                                    if (error>tol):
                                        REPORT_FAILURE(greek, ql.Average.Geometric,
                                                 None, None,
                                                 [],
                                                 payoff, maturity,
                                                 u, q, r, today, v,
                                                 expct, calcl, tol)

                                

# Analytic discrete geometric average-price Asians

In [97]:
dc = ql.Actual360()
today = ql.Settings.instance().evaluationDate
spot = ql.SimpleQuote(100.0)
qRate = ql.SimpleQuote(0.03)
qTS = ql.FlatForward(today, ql.QuoteHandle(qRate), dc)
rRate = ql.SimpleQuote(0.06)
rTS = ql.FlatForward(today, ql.QuoteHandle(rRate), dc)
vol = ql.SimpleQuote(0.20)
volTS = ql.BlackConstantVol(today, ql.NullCalendar(), ql.QuoteHandle(vol), dc)

In [98]:
stochProcess = ql.BlackScholesMertonProcess(ql.QuoteHandle(spot), ql.YieldTermStructureHandle(qTS), 
                                            ql.YieldTermStructureHandle(rTS), 
                                            ql.BlackVolTermStructureHandle(volTS))

In [99]:
engine = ql.AnalyticDiscreteGeometricAveragePriceAsianEngine(stochProcess)

In [100]:
averageType = ql.Average.Geometric

In [101]:
runningAccumulator = 1.0
pastFixings = 0
futureFixings = 10
option_type = ql.Option.Call
strike = 100.0
payoff = ql.PlainVanillaPayoff(option_type, strike)

exerciseDate = today + 360
exercise = ql.EuropeanExercise(exerciseDate)

In [114]:
dt = int(360/futureFixings+0.5)
fixingDates = [today + i*dt for i in range(1, futureFixings+1)]

In [116]:
exerciseDate

Date(7,4,2020)

In [115]:
fixingDates

[Date(19,5,2019),
 Date(24,6,2019),
 Date(30,7,2019),
 Date(4,9,2019),
 Date(10,10,2019),
 Date(15,11,2019),
 Date(21,12,2019),
 Date(26,1,2020),
 Date(2,3,2020),
 Date(7,4,2020)]

In [117]:
option = ql.DiscreteAveragingAsianOption(averageType, runningAccumulator,
                                        pastFixings, fixingDates,
                                        payoff, exercise)

In [118]:
option.setPricingEngine(engine)

In [119]:
option.NPV()

5.342560663499418

In [120]:
calculated = option.NPV()
expected = 5.3425606635
tolerance = 1e-10
if (abs(calculated-expected) > tolerance):
    print("FAILURE")

# Analytic discrete geometric average-strike Asians

In [188]:
dc = ql.Actual360()
today = ql.Settings.instance().evaluationDate
spot = ql.SimpleQuote(100.0)
qRate = ql.SimpleQuote(0.03)
qTS = ql.FlatForward(today, ql.QuoteHandle(qRate), dc)
rRate = ql.SimpleQuote(0.06)
rTS = ql.FlatForward(today, ql.QuoteHandle(rRate), dc)
vol = ql.SimpleQuote(0.20)
volTS = ql.BlackConstantVol(today, ql.NullCalendar(), ql.QuoteHandle(vol), dc)

In [189]:
stochProcess = ql.BlackScholesMertonProcess(ql.QuoteHandle(spot), ql.YieldTermStructureHandle(qTS), 
                                            ql.YieldTermStructureHandle(rTS), 
                                            ql.BlackVolTermStructureHandle(volTS))

In [190]:
engine = ql.AnalyticDiscreteGeometricAverageStrikeAsianEngine(stochProcess)

In [191]:
averageType = ql.Average.Geometric

In [192]:
runningAccumulator = 1.0
pastFixings = 0
futureFixings = 10
option_type = ql.Option.Call
strike = 100.0
payoff = ql.PlainVanillaPayoff(option_type, strike)

exerciseDate = today + 360
exercise = ql.EuropeanExercise(exerciseDate)

In [193]:
dt = int(360/futureFixings+0.5)
fixingDates = [today + i*dt for i in range(1, futureFixings+1)]

In [194]:
option = ql.DiscreteAveragingAsianOption(averageType, runningAccumulator,
                                        pastFixings, fixingDates,
                                        payoff, exercise)

In [195]:
option.setPricingEngine(engine)

In [196]:
option.NPV()

4.971092526326878

In [197]:
calculated = option.NPV()
expected = 4.97109
tolerance = 1e-5
if (abs(calculated-expected) > tolerance):
    print("FAILURE")

# Monte Carlo discrete geometric average-price Asians

In [198]:
tolerance = 4.0e-3

In [213]:
engine = ql.MCDiscreteGeometricAPEngine(stochProcess, "lowdiscrepancy",                                      
                                         brownianBridge=True,
                                         antitheticVariate=False,
                                         requiredSamples=8191,
                                         requiredTolerance=None,
                                         maxSamples=None,
                                         seed=0)

In [214]:
averageType = ql.Average.Geometric

In [215]:
runningAccumulator = 1.0
pastFixings = 0
futureFixings = 10
option_type = ql.Option.Call
strike = 100.0
payoff = ql.PlainVanillaPayoff(option_type, strike)

exerciseDate = today + 360
exercise = ql.EuropeanExercise(exerciseDate)

In [216]:
dt = int(360/futureFixings+0.5)
fixingDates = [today + i*dt for i in range(1, futureFixings+1)]

In [217]:
option = ql.DiscreteAveragingAsianOption(averageType, runningAccumulator,
                                        pastFixings, fixingDates,
                                        payoff, exercise)

In [218]:
option.setPricingEngine(engine)

In [219]:
option.NPV()

5.338879887520727

In [220]:
calculated = option.NPV()

In [221]:
engine2 = ql.AnalyticDiscreteGeometricAveragePriceAsianEngine(stochProcess)

In [222]:
option.setPricingEngine(engine2)

In [223]:
option.NPV()

5.342560663499418

In [224]:
expected = option.NPV()

In [225]:
if (abs(calculated-expected) > tolerance):
    print("FAILURE")

# Monte Carlo discrete arithmetic average-price Asians

In [152]:
cases4 = [
        [ ql.Option.Put, 90.0, 87.0, 0.06, 0.025, 0.0, 11.0/12.0, 2,
          0.13, True, 1.3942835683 ],
        [ ql.Option.Put, 90.0, 87.0, 0.06, 0.025, 0.0, 11.0/12.0, 4,
          0.13, True, 1.5852442983 ],
        [ ql.Option.Put, 90.0, 87.0, 0.06, 0.025, 0.0, 11.0/12.0, 8,
          0.13, True, 1.66970673 ],
        [ ql.Option.Put, 90.0, 87.0, 0.06, 0.025, 0.0, 11.0/12.0, 12,
          0.13, True, 1.6980019214 ],
        [ ql.Option.Put, 90.0, 87.0, 0.06, 0.025, 0.0, 11.0/12.0, 26,
          0.13, True, 1.7255070456 ],
        [ ql.Option.Put, 90.0, 87.0, 0.06, 0.025, 0.0, 11.0/12.0, 52,
          0.13, True, 1.7401553533 ],
        [ ql.Option.Put, 90.0, 87.0, 0.06, 0.025, 0.0, 11.0/12.0, 100,
          0.13, True, 1.7478303712 ],
        [ ql.Option.Put, 90.0, 87.0, 0.06, 0.025, 0.0, 11.0/12.0, 250,
          0.13, True, 1.7490291943 ],
        [ ql.Option.Put, 90.0, 87.0, 0.06, 0.025, 0.0, 11.0/12.0, 500,
          0.13, True, 1.7515113291 ],
        [ ql.Option.Put, 90.0, 87.0, 0.06, 0.025, 0.0, 11.0/12.0, 1000,
          0.13, True, 1.7537344885 ],
        [ ql.Option.Put, 90.0, 87.0, 0.06, 0.025, 1.0/12.0, 11.0/12.0, 2,
          0.13, True, 1.8496053697 ],
        [ ql.Option.Put, 90.0, 87.0, 0.06, 0.025, 1.0/12.0, 11.0/12.0, 4,
          0.13, True, 2.0111495205 ],
        [ ql.Option.Put, 90.0, 87.0, 0.06, 0.025, 1.0/12.0, 11.0/12.0, 8,
          0.13, True, 2.0852138818 ],
        [ ql.Option.Put, 90.0, 87.0, 0.06, 0.025, 1.0/12.0, 11.0/12.0, 12,
          0.13, True, 2.1105094397 ],
        [ ql.Option.Put, 90.0, 87.0, 0.06, 0.025, 1.0/12.0, 11.0/12.0, 26,
          0.13, True, 2.1346526695 ],
        [ ql.Option.Put, 90.0, 87.0, 0.06, 0.025, 1.0/12.0, 11.0/12.0, 52,
          0.13, True, 2.147489651 ],
        [ ql.Option.Put, 90.0, 87.0, 0.06, 0.025, 1.0/12.0, 11.0/12.0, 100,
          0.13, True, 2.154728109 ],
        [ ql.Option.Put, 90.0, 87.0, 0.06, 0.025, 1.0/12.0, 11.0/12.0, 250,
          0.13, True, 2.1564276565 ],
        [ ql.Option.Put, 90.0, 87.0, 0.06, 0.025, 1.0/12.0, 11.0/12.0, 500,
          0.13, True, 2.1594238588 ],
        [ ql.Option.Put, 90.0, 87.0, 0.06, 0.025, 1.0/12.0, 11.0/12.0, 1000,
          0.13, True, 2.1595367326 ],
        [ ql.Option.Put, 90.0, 87.0, 0.06, 0.025, 3.0/12.0, 11.0/12.0, 2,
          0.13, True, 2.63315092584 ],
        [ ql.Option.Put, 90.0, 87.0, 0.06, 0.025, 3.0/12.0, 11.0/12.0, 4,
          0.13, True, 2.76723962361 ],
        [ ql.Option.Put, 90.0, 87.0, 0.06, 0.025, 3.0/12.0, 11.0/12.0, 8,
          0.13, True, 2.83124836881 ],
        [ ql.Option.Put, 90.0, 87.0, 0.06, 0.025, 3.0/12.0, 11.0/12.0, 12,
          0.13, True, 2.84290301412 ],
        [ ql.Option.Put, 90.0, 87.0, 0.06, 0.025, 3.0/12.0, 11.0/12.0, 26,
          0.13, True, 2.88179560417 ],
        [ ql.Option.Put, 90.0, 87.0, 0.06, 0.025, 3.0/12.0, 11.0/12.0, 52,
          0.13, True, 2.88447044543 ],
        [ ql.Option.Put, 90.0, 87.0, 0.06, 0.025, 3.0/12.0, 11.0/12.0, 100,
          0.13, True, 2.89985329603 ],
        [ ql.Option.Put, 90.0, 87.0, 0.06, 0.025, 3.0/12.0, 11.0/12.0, 250,
          0.13, True, 2.90047296063 ],
        [ ql.Option.Put, 90.0, 87.0, 0.06, 0.025, 3.0/12.0, 11.0/12.0, 500,
          0.13, True, 2.89813412160 ],
        [ ql.Option.Put, 90.0, 87.0, 0.06, 0.025, 3.0/12.0, 11.0/12.0, 1000,
          0.13, True, 2.89703362437 ]
   ]

In [153]:
averageType = ql.Average.Arithmetic

    struct DiscreteAverageData {
        0 Option::Type type;
        1 Real underlying;
        2 Real strike;
        3 Rate dividendYield;
        4 Rate riskFreeRate;
        5 Time first;
        6 Time length;
        7 Size fixings;
        8 Volatility volatility;
        9 bool controlVariate;
        10 Real result;
    };

In [235]:
runningSum = 0.00
pastFixings = 0
averageType = ql.Average.Arithmetic

for l in range(0, len(cases4)):
    payoff = ql.PlainVanillaPayoff(cases4[l][0], cases4[l][2])
    dt = cases4[l][6]/(cases4[l][7]-1)
    timeIncrements = [cases4[l][5]]
    fixingDates = [today + int(timeIncrements[0]*360+0.5)]
    for i in range(1, cases4[l][7]) :
        timeIncrements += [i*dt + cases4[l][5]]
        fixingDates += [today + int(timeIncrements[i]*360+0.5)]
    exercise = ql.EuropeanExercise(fixingDates[cases4[l][7]-1])
    
    spot.setValue(cases4[l][1])
    qRate.setValue(cases4[l][3])
    rRate.setValue(cases4[l][4])
    vol.setValue(cases4[l][8])
    
    stochProcess = ql.BlackScholesMertonProcess(ql.QuoteHandle(spot), ql.YieldTermStructureHandle(qTS), 
                                            ql.YieldTermStructureHandle(rTS), 
                                            ql.BlackVolTermStructureHandle(volTS))
    
    engine = ql.MCDiscreteArithmeticAPEngine(stochProcess, "lowdiscrepancy",                                      
                                         brownianBridge=True,
                                         antitheticVariate=False,
                                             controlVariate=cases4[l][9],
                                         requiredSamples=2047,
                                         requiredTolerance=None,
                                         maxSamples=None,
                                         seed=0)
    
    option = ql.DiscreteAveragingAsianOption(averageType, runningSum,
                                            pastFixings, fixingDates,
                                            payoff, exercise)
    
    option.setPricingEngine(engine)
    print(l)
    calculated = option.NPV()
    expected = cases4[l][10]
    tolerance = 2.0e-2
    if (abs(calculated-expected) > tolerance):
        print("FAILURE")
        print(l)
    
    if(cases4[l][7] < 100):
        engine = ql.FdBlackScholesAsianEngine(stochProcess, 100, 100, 100)
        option.setPricingEngine(engine)
        calculated = option.NPV()
        if (abs(calculated-expected) > tolerance):
            print("FAILURE")
            print(l)
        

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29


# Monte Carlo discrete arithmetic average-strike Asians

In [236]:
cases5 = [
        [ ql.Option.Call, 90.0, 87.0, 0.06, 0.025, 0.0, 11.0/12.0, 2,
          0.13, True, 1.51917595129 ],
        [ ql.Option.Call, 90.0, 87.0, 0.06, 0.025, 0.0, 11.0/12.0, 4,
          0.13, True, 1.67940165674 ],
        [ ql.Option.Call, 90.0, 87.0, 0.06, 0.025, 0.0, 11.0/12.0, 8,
          0.13, True, 1.75371215251 ],
        [ ql.Option.Call, 90.0, 87.0, 0.06, 0.025, 0.0, 11.0/12.0, 12,
          0.13, True, 1.77595318693 ],
        [ ql.Option.Call, 90.0, 87.0, 0.06, 0.025, 0.0, 11.0/12.0, 26,
          0.13, True, 1.81430536630 ],
        [ ql.Option.Call, 90.0, 87.0, 0.06, 0.025, 0.0, 11.0/12.0, 52,
          0.13, True, 1.82269246898 ],
        [ ql.Option.Call, 90.0, 87.0, 0.06, 0.025, 0.0, 11.0/12.0, 100,
          0.13, True, 1.83822402464 ],
        [ ql.Option.Call, 90.0, 87.0, 0.06, 0.025, 0.0, 11.0/12.0, 250,
          0.13, True, 1.83875059026 ],
        [ ql.Option.Call, 90.0, 87.0, 0.06, 0.025, 0.0, 11.0/12.0, 500,
          0.13, True, 1.83750703638 ],
        [ ql.Option.Call, 90.0, 87.0, 0.06, 0.025, 0.0, 11.0/12.0, 1000,
          0.13, True, 1.83887181884 ],
        [ ql.Option.Call, 90.0, 87.0, 0.06, 0.025, 1.0/12.0, 11.0/12.0, 2,
          0.13, True, 1.51154400089 ],
        [ ql.Option.Call, 90.0, 87.0, 0.06, 0.025, 1.0/12.0, 11.0/12.0, 4,
          0.13, True, 1.67103508506 ],
        [ ql.Option.Call, 90.0, 87.0, 0.06, 0.025, 1.0/12.0, 11.0/12.0, 8,
          0.13, True, 1.74529684070 ],
        [ ql.Option.Call, 90.0, 87.0, 0.06, 0.025, 1.0/12.0, 11.0/12.0, 12,
          0.13, True, 1.76667074564 ],
        [ ql.Option.Call, 90.0, 87.0, 0.06, 0.025, 1.0/12.0, 11.0/12.0, 26,
          0.13, True, 1.80528400613 ],
        [ ql.Option.Call, 90.0, 87.0, 0.06, 0.025, 1.0/12.0, 11.0/12.0, 52,
          0.13, True, 1.81400883891 ],
        [ ql.Option.Call, 90.0, 87.0, 0.06, 0.025, 1.0/12.0, 11.0/12.0, 100,
          0.13, True, 1.82922901451 ],
        [ ql.Option.Call, 90.0, 87.0, 0.06, 0.025, 1.0/12.0, 11.0/12.0, 250,
          0.13, True, 1.82937111773 ],
        [ ql.Option.Call, 90.0, 87.0, 0.06, 0.025, 1.0/12.0, 11.0/12.0, 500,
          0.13, True, 1.82826193186 ],
        [ ql.Option.Call, 90.0, 87.0, 0.06, 0.025, 1.0/12.0, 11.0/12.0, 1000,
          0.13, True, 1.82967846654 ],
        [ ql.Option.Call, 90.0, 87.0, 0.06, 0.025, 3.0/12.0, 11.0/12.0, 2,
          0.13, True, 1.49648170891 ],
        [ ql.Option.Call, 90.0, 87.0, 0.06, 0.025, 3.0/12.0, 11.0/12.0, 4,
          0.13, True, 1.65443100462 ],
        [ ql.Option.Call, 90.0, 87.0, 0.06, 0.025, 3.0/12.0, 11.0/12.0, 8,
          0.13, True, 1.72817806731 ],
        [ ql.Option.Call, 90.0, 87.0, 0.06, 0.025, 3.0/12.0, 11.0/12.0, 12,
          0.13, True, 1.74877367895 ],
        [ ql.Option.Call, 90.0, 87.0, 0.06, 0.025, 3.0/12.0, 11.0/12.0, 26,
          0.13, True, 1.78733801988 ],
        [ ql.Option.Call, 90.0, 87.0, 0.06, 0.025, 3.0/12.0, 11.0/12.0, 52,
          0.13, True, 1.79624826757 ],
        [ ql.Option.Call, 90.0, 87.0, 0.06, 0.025, 3.0/12.0, 11.0/12.0, 100,
          0.13, True, 1.81114186876 ],
        [ ql.Option.Call, 90.0, 87.0, 0.06, 0.025, 3.0/12.0, 11.0/12.0, 250,
          0.13, True, 1.81101152587 ],
        [ ql.Option.Call, 90.0, 87.0, 0.06, 0.025, 3.0/12.0, 11.0/12.0, 500,
          0.13, True, 1.81002311939 ],
        [ ql.Option.Call, 90.0, 87.0, 0.06, 0.025, 3.0/12.0, 11.0/12.0, 1000,
          0.13, True, 1.81145760308 ]
    ]

In [241]:
runningSum = 0.00
pastFixings = 0
averageType = ql.Average.Arithmetic

for l in range(0, len(cases5)):
    payoff = ql.PlainVanillaPayoff(cases5[l][0], cases5[l][2])
    dt = cases5[l][6]/(cases5[l][7]-1)
    timeIncrements = [cases5[l][5]]
    fixingDates = [today + int(timeIncrements[0]*360+0.5)]
    for i in range(1, cases5[l][7]) :
        timeIncrements += [i*dt + cases5[l][5]]
        fixingDates += [today + int(timeIncrements[i]*360+0.5)]
    exercise = ql.EuropeanExercise(fixingDates[cases5[l][7]-1])
    
    spot.setValue(cases5[l][1])
    qRate.setValue(cases5[l][3])
    rRate.setValue(cases5[l][4])
    vol.setValue(cases5[l][8])
    
    stochProcess = ql.BlackScholesMertonProcess(ql.QuoteHandle(spot), ql.YieldTermStructureHandle(qTS), 
                                            ql.YieldTermStructureHandle(rTS), 
                                            ql.BlackVolTermStructureHandle(volTS))
    
    engine = ql.MCDiscreteArithmeticASEngine(stochProcess, "lowdiscrepancy",                                      
                                         brownianBridge=True,
                                         antitheticVariate=False,                                             
                                         requiredSamples=1023,
                                         requiredTolerance=None,
                                         maxSamples=None,
                                         seed=3456789)
    
    option = ql.DiscreteAveragingAsianOption(averageType, runningSum,
                                            pastFixings, fixingDates,
                                            payoff, exercise)
    
    option.setPricingEngine(engine)
    print(l)
    calculated = option.NPV()
    expected = cases5[l][10]
    tolerance = 2.0e-2
    if (abs(calculated-expected) > tolerance):
        print("FAILURE")
        print(l)

        

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
